In [38]:
import simpleCongurentGenerators
import time
import numpy as np

In [72]:
sample_size = int(1e6)

In [36]:
## performance check (seconds - ok for educationals purpose)
start = time.time()
M=13
sample_lcg = simpleCongurentGenerators.get_n_lcg(M, 1, 5, 2, size=int(1e6))
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 2.2014315128326416


## Map PRNG sample on [0,1] and perform test 

In [45]:
sample_lcg_mapped = sample_lcg / M
assert np.all((sample_lcg_mapped) >= 0 & (sample_lcg_mapped <= 1)) , "not all elements in (0,1)"

In [48]:
from chi_square_test import chi_square_test

In [51]:
test_stat = chi_square_test(sample_lcg_mapped)
test_stat


1420.097041599997

In [52]:
from scipy import stats

In [55]:
stats.kstest(sample_lcg_mapped, 'uniform')

KstestResult(statistic=0.07692346153846152, pvalue=0.0, statistic_location=0.5384615384615384, statistic_sign=1)

## RC(32)

In [61]:
from PRGA import PRGA
from KSA import Keys, KSA

In [74]:
keys = Keys(np.array([3, 4, 2, 5]))
ksa = KSA(m=32)
ksa.schedule(keys)
prga = PRGA(ksa)
start = time.time()
sample_rc = prga.get_random(sample_size)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 2.2094829082489014


In [76]:
sample_rc_mapped = sample_rc / ksa.m
assert np.all((sample_rc_mapped) >= 0 & (sample_rc_mapped <= 1)) , "not all elements in (0,1)"